In [1]:
%matplotlib inline

In [2]:
import os

import numpy as np
import loader as L

import matplotlib.pyplot as plt

from scipy.interpolate import griddata, interp1d

# from __future__ import print_function # (not needed in python3)

from scipy.ndimage import rotate

Extracting EDF parameters from /home/ced/Coding/EEGLearn/data_gdf/A01T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...


/home/ced/.local/lib/python3.5/site-packages/mne/io/edf/edf.py:1083: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]
/home/ced/.local/lib/python3.5/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Overlapping events detected. Use find_edf_events for the original events.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')
/home/ced/.local/lib/python3.5/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Interpolating stim channel. Events may jitter.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')
/home/ced/.local/lib/python3.5/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')


Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


In [4]:
images = interpolate(locs, features, gridsize)

Interpolating 288/288

In [3]:

locs = [[ 35.5,  -0.1 ],    # Fz
        [ 12.2,  56.16],    # FC3
        [  9.9,  27.6 ],    # FC1
        [  9.4,  -0.1 ],    # FCz
        [ 10.5, -28.32],    # FC2
        [ 13  , -57   ],    # FC4
        [-15.1,  87.48],    # C5
        [-16.2,  57.48],    # C3
        [-16.5,  28.44],    # C1
        [-16.8, - 0.1 ],    # Cz
        [-16.3, -29.64],    # C2
        [-15.5, -58.68],    # C4
        [-14.1, -87.6 ],    # C6
        [-40.9,  53.16],    # CP3
        [-39.3,  26.4 ],    # CP1
        [-39.3, - 0.12],    # CPz
        [-38.9, -27.84],    # CP2
        [-40.2, -54.24],    # CP4
        [-65.5,  22.08],    # P1
        [-63.8, - 0.12],    # Pz
        [-65.1, -23.28],    # P2
        [-79.7, - 0.1 ]]    # POz

locs = np.array(locs)

channel_labels = ['Fz'
        , 'FC3'
        , 'FC1'
        , 'FCz'
        , 'FC2'
        , 'FC4'
        , 'C5'
        , 'C3'
        , 'C1'
        , 'Cz'
        , 'C2'
        , 'C4'
        , 'C6'
        , 'CP3'
        , 'CP1'
        , 'CPz'
        , 'CP2'
        , 'CP4'
        , 'P1'
        , 'Pz'
        , 'P3'
        , 'POz'
        ]


gridsize = 32



features = L.train_set.X
labels = L.train_set.y


In [4]:
# def gen_images(locs, features, n_gridpoints, normalize=True,
#                augment=False, pca=False, std_mult=0.1, n_components=2, edgeless=False):
# needs defined: locs, features, n_gridpoints

def interpolate(locs, features, n_gridpoints):
    """
    Generates EEG images given electrode locations in 2D space and multiple feature values for each electrode

    :param locs: An array with shape [n_electrodes, 2] containing X, Y coordinates for each electrode.
    :param features: Feature matrix as [n_samples, n_features]
                        Features are as columns.
                        Features corresponding to each frequency band are concatenated.
                        (alpha1, alpha2, ..., beta1, beta2,...)
    :param n_gridpoints: Number of pixels in the output images
    :return:            Tensor of size [samples, colors, W, H] containing generated
                        images.
    """
    
    cut = lambda a: a if len(a.shape) < 3 else a[:, :, 0]
    
    feat_array_temp = []
    nElectrodes = locs.shape[0]     # Number of electrodes
    # Test whether the feature vector length is divisible by number of electrodes
    assert features.shape[1] % nElectrodes == 0
    n_colors = features.shape[1] / nElectrodes
    for c in range(n_colors):
        feat_array_temp.append(cut(features[:, c * nElectrodes : nElectrodes * (c+1)]))
    nSamples = features.shape[0]
    # getting some surrounding values right
    grid_x, grid_y = np.mgrid[
                     min(locs[:, 0]):max(locs[:, 0]):n_gridpoints*1j,
                     min(locs[:, 1]):max(locs[:, 1]):n_gridpoints*1j
                     ]
    temp_interp = []
    for c in range(n_colors):
        temp_interp.append(np.zeros([nSamples, n_gridpoints, n_gridpoints]))
    # Interpolating
    for i in xrange(nSamples):
        for c in range(n_colors):
            temp_interp[c][i, :, :] = cut(griddata(locs, feat_array_temp[c][i, :], (grid_x, grid_y),
                                    method='cubic', fill_value=np.nan))
    #                                 method='cubic', fill_value=np.nan)# [:, :, 0]
    #                                 method='cubic', fill_value=np.nan)
        print('Interpolating {0}/{1}\r'.format(i+1, nSamples), end='\r')
    images = np.swapaxes(np.asarray(temp_interp), 0, 1)     # swap axes to have [samples, colors, W, H]
    return images

In [28]:
features.shape

(288, 22, 1125)

In [35]:
mx = interp1d([-85, 42], [0, gridsize])
my = interp1d([-90, 108], [0, gridsize])

In [9]:
images2 = images[:]

In [16]:
for i in xrange(images.shape[0]):
    print('Saving {0}/{1}\r'.format(i+1, images.shape[0]), end='\r')
    # for i in xrange(22):
    #     plt.text(my(locs[i][1]), mx(locs[i][0]), channel_labels[i])
    plt.imshow(rotate(images[i,0,:,:], 45, cval=np.nan), cmap='RdYlBu')
    plt.savefig(os.path.join('images4/','%d.png' % i))
    plt.close()

Saving 288/288

/usr/lib/python2.7/dist-packages/matplotlib/colors.py:941: UserWarning: Warning: converting a masked element to nan.
  vmin = float(vmin)
/usr/lib/python2.7/dist-packages/matplotlib/colors.py:942: UserWarning: Warning: converting a masked element to nan.
  vmax = float(vmax)


In [22]:
i = 0
plt.imshow(rotate(images[i,0,:,:], 45, cval=np.nan, prefilter=False), cmap='RdYlBu')
plt.savefig(os.path.join('images4/','%d.png' % i))
plt.show()
plt.close()
plt.imshow(images[i,0,:,:], cmap='RdYlBu')
plt.show()
plt.close()
# plt.savefig(os.path.join('images4/','%d.png' % i))
# plt.close()